In [1]:
import random
import pandas as pd
import Covid19_model as CM
import numpy as np
import shutil
import sys
import os.path
import math
import csv
import lhsmdu
import time
from Pyomo_solve import solve_ode

# Set parameter

In [2]:
D = 1e2
n_site = 8
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 1.8e6
length = 50

In [3]:
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
seed = 0
env = CM.Env_model(init_state, deter_para, in_deter_para, seed)
env.set_para_truth(para_truth)

In [5]:
tic = time.time()
random.seed(seed)
observation, state = init_state.copy(), init_state.copy()
state_buffer, new_buffer  = np.zeros([T, 6]), np.zeros([T, 4])
score, ni, ne = 0, 0, 0
done = False
index = [0]
start_time = 6
for t in range(T):
    p_set = env.para_sampling(100)
    para = p_set[index[0]]
    action_buffer = [] 
    reward_buffer = []
    s = np.array(state)
    ac = []
    if t > start_time:
        for a in range(n_site):
            try:
                aa = solve_ode(s[:,a], np.sum(s[1:-1,a])*budget/np.sum(s[1:-1,:]), para, length)
            except Exception:
                print("pyomo fail")
            ac.append(aa)
        action = np.array(ac).T
        if budget < 0: 
            action = np.zeros((5,n_site))
    else:
        action = np.zeros((5,n_site))
        
    
    next_state, obs, reward, cost, new_i, new_e, ck = env.online_state_estimate(state, observation, 
                                                                                action, p_set, 100, 1)
    #if not done and t > start_time: 
     #   env.space_update(ck, p_set, n_select)
    index = ck.argsort()[0][:1]
    score += reward
    budget -= cost
    ni += new_i
    ne += new_e

    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'True State Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, score:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]),
                  budget, score))
    print('\r', 'Observe Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]),
                  np.sum(state[4]), np.sum(state[5]))) 
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time() - tic)

 True State Time: 0 | S: 18118944.0, E: 56.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 1800000.00, score:-114.64
 Observe Time: 0 | S: 18118944.0, E: 56.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0
 Time: 0 | new_e:26.5, cum_e:26.5, new_i:23.6, cum_i:23.6
 True State Time: 1 | S: 18118899.9, E: 73.9, A: 0.0, I: 23.6, D: 0.0, R: 2.5, Budget: 1800000.00, score:-277.18
 Observe Time: 1 | S: 18118901.0, E: 72.0, A: 0.0, I: 23.6, D: 0.0, R: 0.0
 Time: 1 | new_e:37.1, cum_e:63.6, new_i:32.7, cum_i:56.3
 True State Time: 2 | S: 18118838.0, E: 99.8, A: 0.0, I: 56.3, D: 0.0, R: 5.9, Budget: 1800000.00, score:-508.20
 Observe Time: 2 | S: 18118841.0, E: 100.0, A: 0.0, I: 56.3, D: 0.0, R: 0.0
 Time: 2 | new_e:52.5, cum_e:116.1, new_i:45.7, cum_i:102.0
 True State Time: 3 | S: 18118750.6, E: 137.0, A: 0.0, I: 101.9, D: 0.1, R: 10.4, Budget: 1800000.00, score:-837.11
 Observe Time: 3 | S: 18118756.0, E: 138.0, A: 0.0, I: 101.9, D: 0.1, R: 6.0
 Time: 3 | new_e:74.4, cum_e:190.4, new_i:64.2, cum_i:166.2
 True St

 True State Time: 23 | S: 17278654.3, E: 9.7, A: 140.9, I: 413.7, D: 2.6, R: 839778.8, Budget: 1161996.96, score:-5731.29
 Observe Time: 23 | S: 17278705.0, E: 9.0, A: 140.9, I: 413.7, D: 2.6, R: 839779.0
 Time: 23 | new_e:4.4, cum_e:1285.9, new_i:19.1, cum_i:1706.8
 True State Time: 24 | S: 17231150.0, E: 4.4, A: 127.7, I: 381.0, D: 2.7, R: 887334.4, Budget: 1142412.72, score:-5742.55
 Observe Time: 24 | S: 17231200.0, E: 1.0, A: 127.7, I: 381.0, D: 2.7, R: 887333.0
 Time: 24 | new_e:2.0, cum_e:1287.9, new_i:15.4, cum_i:1722.2
 True State Time: 25 | S: 17198313.1, E: 3.6, A: 110.2, I: 348.7, D: 2.7, R: 920221.6, Budget: 1125153.05, score:-5752.11
 Observe Time: 25 | S: 17198365.0, E: 0.0, A: 110.2, I: 348.7, D: 2.7, R: 920220.0
 Time: 25 | new_e:1.6, cum_e:1289.5, new_i:13.3, cum_i:1735.5
 True State Time: 26 | S: 17169681.0, E: 3.4, A: 94.7, I: 318.4, D: 2.7, R: 948899.8, Budget: 1103530.89, score:-5760.96
 Observe Time: 26 | S: 17169735.0, E: 0.0, A: 94.7, I: 318.4, D: 2.7, R: 94889

 True State Time: 46 | S: 16557149.7, E: 6.4, A: 10.6, I: 53.1, D: 3.1, R: 1561777.1, Budget: 682657.22, score:-5913.14
 Observe Time: 46 | S: 16557232.0, E: 2.0, A: 10.6, I: 53.1, D: 3.1, R: 1561775.0
 Time: 46 | new_e:2.7, cum_e:1321.1, new_i:3.6, cum_i:1835.7
 True State Time: 47 | S: 16524212.6, E: 3.8, A: 13.3, I: 50.1, D: 3.1, R: 1594717.1, Budget: 663726.16, score:-5920.52
 Observe Time: 47 | S: 16524298.0, E: 0.0, A: 13.3, I: 50.1, D: 3.1, R: 1594716.0
 Time: 47 | new_e:1.6, cum_e:1322.7, new_i:2.9, cum_i:1838.6
 True State Time: 48 | S: 16502862.6, E: 3.0, A: 13.1, I: 46.8, D: 3.1, R: 1616071.4, Budget: 645147.70, score:-5926.37
 Observe Time: 48 | S: 16502950.0, E: 0.0, A: 13.1, I: 46.8, D: 3.1, R: 1616070.0
 Time: 48 | new_e:1.3, cum_e:1324.0, new_i:2.6, cum_i:1841.1
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 49 

 True State Time: 68 | S: 16032297.0, E: 16.1, A: 9.9, I: 37.4, D: 3.3, R: 2086636.3, Budget: 347438.71, score:-6211.80
 Observe Time: 68 | S: 16032464.0, E: 0.0, A: 9.9, I: 37.4, D: 3.3, R: 2086634.0
 Time: 68 | new_e:7.8, cum_e:1388.3, new_i:8.0, cum_i:1918.6
 True State Time: 69 | S: 16021689.7, E: 19.5, A: 10.3, I: 42.0, D: 3.3, R: 2097235.2, Budget: 335254.44, score:-6246.99
 Observe Time: 69 | S: 16021868.0, E: 0.0, A: 10.3, I: 42.0, D: 3.3, R: 2097231.0
 Time: 69 | new_e:8.0, cum_e:1396.3, new_i:8.6, cum_i:1927.2
 True State Time: 70 | S: 16000069.5, E: 22.6, A: 10.7, I: 45.3, D: 3.3, R: 2118848.6, Budget: 323962.73, score:-6287.66
 Observe Time: 70 | S: 16000262.0, E: 0.0, A: 10.7, I: 45.3, D: 3.3, R: 2118844.0
 Time: 70 | new_e:9.3, cum_e:1405.6, new_i:9.8, cum_i:1937.0
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 71

In [1]:
(time.time() - tic) / ( T - start_time + 1)

54.940759860768